# Chatbot using Tools

Chatbot for an airline to look up prices using Tools

In [14]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
import gradio as gr

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-'):
    print("API key looks good so far")
else:
    print("There might be a problem with your API key?")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [15]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [16]:
# Define the get_ticket_price function for the Tool

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

tools = [{"type": "function", "function": price_function}]


In [17]:
# Create the chat function and provide it with the Tools

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(message)
        response, city = handle_tool_call(message)
        print(response)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content


def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [18]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_fUUOZQmQlTx4HtUIHcqhpB3l', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')])
Tool get_ticket_price called for London
{'role': 'tool', 'content': '{"destination_city": "London", "price": "$799"}', 'tool_call_id': 'call_fUUOZQmQlTx4HtUIHcqhpB3l'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_UfonxdVcXSw7s6TnKP0wf7tJ', function=Function(arguments='{"destination_city":"Brisbane"}', name='get_ticket_price'), type='function')])
Tool get_ticket_price called for Brisbane
{'role': 'tool', 'content': '{"destination_city": "Brisbane", "price": "Unknown"}', 'tool_call_id': 'call_UfonxdVcXSw7s6TnKP0wf7tJ'}
